# Build a Customer Support Question Answering Chatbot

First we scrape some content from online articles, we split them into small chunks, compute their embeddings and store them in Deep Lake. Then, we use a user query to retrieve the most relevant chunks from Deep Lake, we put them into a prompt, which will be used to generate the final answer by the LLM.

It is important to note that there is always a risk of generating hallucinations or false information when using LLMs. Although this might not be acceptable for many customers support use cases, the chatbot can still be helpful for assisting operators in drafting answers that they can double-check before sending them to the user.

In the next steps, we'll explore how to manage conversations with GPT-3 and provide examples to demonstrate the effectiveness of this workflow:

First, set up the `OPENAI_API_KEY` and `ACTIVELOOP_TOKEN` environment variables with your API keys and tokens.

As we’re going to use the SeleniumURLLoader LangChain class, and it uses the unstructured and selenium Python library, let’s install it using pip. It is recommended to install the latest version of the library. Nonetheless, please be aware that the code has been tested specifically on version `0.7.7`.

In [ ]:
%pip install unstructured selenium
%pip install langchain==0.0.208 deeplake==3.9.27 openai==0.27.8 tiktoken

In [4]:
import os
from google.colab import userdata
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import DeepLake
from langchain.text_splitter import CharacterTextSplitter
from langchain import OpenAI
from langchain.document_loaders import SeleniumURLLoader
from langchain import PromptTemplate

os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')
activeloop_token = userdata.get('ACTIVELOOP_TOKEN')
os.environ["ACTIVELOOP_TOKEN"] = activeloop_token

The database for our chatbot will consist of articles regarding technical issues.

In [2]:
# we'll use information from the following articles
urls = ['https://beebom.com/what-is-nft-explained/',
        'https://beebom.com/how-delete-spotify-account/',
        'https://beebom.com/how-download-gif-twitter/',
        'https://beebom.com/how-use-chatgpt-linux-terminal/',
        'https://beebom.com/how-delete-spotify-account/',
        'https://beebom.com/how-save-instagram-story-with-music/',
        'https://beebom.com/how-install-pip-windows/',
        'https://beebom.com/how-check-disk-usage-linux/']

### 1: Split the documents into chunks and compute their embeddings
We load the documents from the provided URLs and split them into chunks using the `CharacterTextSplitter` with a chunk size of 1000 and no overlap:


In [3]:
# use the selenium scraper to load the documents
loader = SeleniumURLLoader(urls=urls)
docs_not_splitted = loader.load()

# we split the documents into smaller chunks
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(docs_not_splitted)

Next, we compute the embeddings using `OpenAIEmbeddings` and store them in a Deep Lake vector store on the cloud. In an ideal production scenario, we could upload a whole website or course lesson on a Deep Lake dataset, allowing for search among even thousands or millions of documents. As we are using a cloud serverless Deep Lake dataset, applications running on different locations can easily access the same centralized dataset without the need of deploying a vector store on a custom machine.

Let’s now modify the following code by adding your Activeloop organization ID. It worth noting that the org id is your username by default.

In [ ]:
# Before executing the following code, make sure to have
# your OpenAI key saved in the “OPENAI_API_KEY” environment variable.
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")

# create Deep Lake dataset
# TODO: use your organization id here. (by default, org id is your username)
activeloop_org_id = userdata.get('ACTIVELOOP_ORG_ID')
os.environ["ACTIVELOOP_ORG_ID"] = activeloop_org_id
my_activeloop_org_id = activeloop_org_id
my_activeloop_dataset_name = "langchain_course_customer_support"
dataset_path = f"hub://{my_activeloop_org_id}/{my_activeloop_dataset_name}"
db = DeepLake(dataset_path=dataset_path, embedding_function=embeddings)

# add documents to our Deep Lake dataset
db.add_documents(docs)

To retrieve the most similar chunks to a given query, we can use the `similarity_search` method of the Deep Lake vector store:

In [6]:
# let's see the top relevant documents to a specific query
query = "how to check disk usage in linux?"
docs = db.similarity_search(query)
print(docs[0].page_content)

Home > Tech > How to Check Disk Usage in Linux (4 Methods)

How to Check Disk Usage in Linux (4 Methods)

Beebom Staff

Updated: December 19, 2023

Comments 0

Share

Copied

There may be times when you need to download some important files or transfer some photos to your Linux system, but face a problem of insufficient disk space. You head over to your file manager to delete the large files which you no longer require, but you have no clue which of them are occupying most of your disk space. In this article, we will show some easy methods to check disk usage in Linux from both the terminal and the GUI application.

Table of Contents

Check Disk Space Using the df Command

In Linux, there are many commands to check disk usage, the most common being the df command. The df stands for “Disk Filesystem” in the command, which is a handy way to check the current disk usage and the available disk space in Linux. The syntax for the df command in Linux is as follows:

df <options> <file_system>

### 2: Craft a prompt for GPT-3 using the suggested strategies
We will create a prompt template that incorporates role-prompting, relevant Knowledge Base information, and the user's question:

In [7]:
# let's write a prompt for a customer support chatbot that
# answer questions using information extracted from our db
template = """You are an exceptional customer support chatbot that gently answer questions.

You know the following context information.

{chunks_formatted}

Answer to the following question from a customer. Use only information from the previous context information. Do not invent stuff.

Question: {query}

Answer:"""

prompt = PromptTemplate(
    input_variables=["chunks_formatted", "query"],
    template=template,
)

The template sets the chatbot's persona as an exceptional customer support chatbot. The template takes two input variables: `chunks_formatted`, which consists of the pre-formatted chunks from articles, and `query`, representing the customer's question. The objective is to generate an accurate answer using only the provided chunks without creating any false or invented information.

### 3: Use the GPT3 model with a temperature of 0 for text generation
To generate a response, we first retrieve the top-k (e.g., top-3) chunks most similar to the user query, format the prompt, and send the formatted prompt to the GPT3 model with a temperature of 0.

In [8]:
# the full pipeline

# user question
query = "How to check disk usage in linux?"

# retrieve relevant chunks
docs = db.similarity_search(query)
retrieved_chunks = [doc.page_content for doc in docs]

# format the prompt
chunks_formatted = "\n\n".join(retrieved_chunks)
prompt_formatted = prompt.format(chunks_formatted=chunks_formatted, query=query)

# generate answer
llm = OpenAI(model="gpt-3.5-turbo-instruct", temperature=0)
answer = llm(prompt_formatted)
print(answer)

 There are several methods to check disk usage in Linux. One of the most common methods is by using the df command, which stands for "Disk Filesystem". This command allows you to check the current disk usage and available disk space on your system. You can also use GUI tools such as the GDU Disk Usage Analyzer and the Gnome Disks Tool to monitor disk usage. Additionally, you can use the du command to display the disk usage for a specific directory or compare the disk usage of two directories. Finally, you can use a combination of commands to list out the files and directories occupying the most space and choose to delete them to free up storage space on your computer.
